# 04 Census API Microdata Fetch

Fetch county-level ACS data and build a micro dataset.


## Table of Contents
- [Browse variables](#browse-variables)
- [Fetch county data](#fetch-county-data)
- [Derived rates](#derived-rates)
- [Save processed data](#save-processed-data)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Data notebooks build the datasets used everywhere else. If these steps are wrong, every model result is suspect.
You will practice:
- fetching cross-sectional data from the Census API,
- validating geographic identifiers (FIPS codes),
- computing derived rates from raw counts.


## Prerequisites (Quick Self-Check)
- Completed Part 00 (foundations): environment setup and data loading patterns.
- Census API key set (`CENSUS_API_KEY`) for live fetches (sample data works offline).

## What You Will Produce
- data/processed/census_county_<year>.csv

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.
- You can point to the concrete deliverable(s) listed below and explain how they were produced.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Storing FIPS codes as integers (drops leading zeros, breaks merges).
- Mismatching numerator and denominator universes when computing rates (e.g., using total population instead of civilian non-institutionalized population for insurance rates).
- Forgetting that Census API returns strings — cast to numeric with `pd.to_numeric(..., errors='coerce')`.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook's `data/sample/*` fallback.
- If a rate is outside [0, 1], check your numerator/denominator columns — you may have the wrong ACS variable code.
- If a merge drops rows, check that FIPS codes are 5-character zero-padded strings in both DataFrames.

## Matching Guide
- `docs/guides/01_data/04_census_api_microdata_fetch.md`


## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/01_data/04_census_api_microdata_fetch.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Build a county-level micro dataset from the US Census ACS API.

### Why this matters
This micro track is deliberately different from macro time series:
- observations are counties (not time)
- regression interpretation focuses on cross-sectional relationships
- robust SE (HC3) is usually more relevant than time-series HAC



## Primer: Working with Census ACS data (FIPS codes, variable naming, type casting)

Census ACS data has its own patterns that differ from macro time series.

### 1) FIPS codes (the geographic key for everything)

Every county has a 5-digit FIPS code: 2-digit state + 3-digit county.

```python
# WRONG: FIPS as integer (drops leading zeros)
fips_int = 1001   # intended: Autauga County, AL
# Merge with another dataset that has "01001" → no match!

# RIGHT: FIPS as zero-padded string
fips_str = "01001"  # always 5 characters
# Or from separate columns:
fips = state_fips.str.zfill(2) + county_fips.str.zfill(3)
```

**Rule:** Always store FIPS as strings. After any load or merge, assert they are 5 characters.

### 2) ACS variable naming convention

Variables follow the pattern `TABLE_SEQE` or `TABLE_SEQM`:
- `B01003_001E` → table B01003, sequence 001, **E**stimate
- `B01003_001M` → same variable, **M**argin of error

The `E` suffix is the point estimate; the `M` suffix is the 90% margin of error.

### 3) Type casting (the API returns strings)

The Census API returns all values as strings, including numeric ones. The sentinel value `-666666666` means suppressed/unavailable.

```python
import pandas as pd

# Cast to numeric, turning non-numeric strings (like sentinels) into NaN
df["B01003_001E"] = pd.to_numeric(df["B01003_001E"], errors="coerce")
```

**Always** cast numeric columns immediately after loading. If you skip this, arithmetic operations will silently fail or produce wrong results.

### 4) Derived rates: numerator ÷ denominator

When computing rates (poverty rate, uninsured rate), check that:
- numerator and denominator come from the same "universe" (e.g., civilian non-institutionalized population)
- you handle division by zero (small counties may have zero in a denominator category)

```python
import numpy as np

# Safe division: replace 0 with NaN to avoid inf
rate = numerator / denominator.replace({0: np.nan})
assert (rate.dropna().between(0, 1)).all(), "Rate out of bounds — check your variables"
```

### 5) Merging ACS tables

When combining variables from different ACS tables:
```python
merged = table1.merge(table2, on="fips", validate="1:1")
assert len(merged) == len(table1), "Merge changed row count — check FIPS alignment"
```

Use `validate="1:1"` to catch unexpected duplicates or missing keys.


<a id="browse-variables"></a>
## Browse variables

### Goal
Learn how ACS variable codes work and choose a starter set.

We'll focus on a practical starter set:
- population
- median household income
- median gross rent
- median home value
- poverty count (to build a poverty rate)
- labor force / unemployment (to build an unemployment rate)



### Your Turn (1): Fetch or load variables.json


In [ ]:
import json
from src import census_api

year = 2022  # TODO: change if you want a different year
raw_dir = RAW_DIR / 'census'
raw_dir.mkdir(parents=True, exist_ok=True)
vars_path = raw_dir / f'variables_{year}.json'

# TODO: Load variables metadata.
# - If vars_path exists, load it from disk.
# - Otherwise, fetch from the API and save it to vars_path.
...



### Your Turn (2): Search for relevant variables


In [ ]:
# The variables metadata is a nested JSON structure.
# TODO: Explore it and search for keywords like:
# - 'Median household income'
# - 'Median gross rent'
# - 'Poverty'
# - 'Labor force'

# Hint: variables are typically under payload['variables'].
...



<a id="fetch-county-data"></a>
## Fetch county data

### Goal
Fetch a county-level table for your chosen variables.

Default geography:
- all counties: `for=county:*`
- within all states: `in=state:*`



### Your Turn (1): Choose a starter variable set


In [ ]:
# TODO: Use a starter set.
# These are commonly-used ACS 5-year estimate codes:
acs_vars = [
    'NAME',
    'B01003_001E',  # total population
    'B19013_001E',  # median household income
    'B25064_001E',  # median gross rent
    'B25077_001E',  # median home value
    'B17001_002E',  # count below poverty level
    'B23025_002E',  # in labor force
    'B23025_005E',  # unemployed
]

acs_vars



### Your Turn (2): Fetch the ACS table


In [ ]:
import pandas as pd
from src import census_api

# TODO: Fetch the data from the API.
# Hint: census_api.fetch_acs(year=..., get=..., for_geo='county:*', in_geo='state:*')
try:
    df_raw = census_api.fetch_acs(year=year, get=acs_vars, for_geo='county:*', in_geo='state:*')
except Exception as exc:
    df_raw = None
    print('Fetch failed, will use sample. Error:', exc)

df_raw.head() if df_raw is not None else None



### Your Turn (3): Fallback to sample


In [ ]:
import pandas as pd

# TODO: If df_raw is None, load the sample dataset.
if df_raw is None:
    df_raw = pd.read_csv(SAMPLE_DIR / 'census_county_sample.csv')

df_raw.head()



<a id="derived-rates"></a>
## Derived rates

### Goal
Turn raw counts into rates (more comparable across counties).

You will build:
- unemployment_rate = unemployed / labor_force
- poverty_rate = below_poverty / population



### Your Turn (1): Cast numeric columns


In [ ]:
# TODO: Ensure numeric columns are numeric (some API returns strings).
# Hint: pd.to_numeric(..., errors='coerce')
...



### Your Turn (2): Build derived rates safely


In [ ]:
import numpy as np

# TODO: Compute rates with safe division.
# Replace division-by-zero with NaN.

pop = df_raw['B01003_001E'].astype(float)
labor_force = df_raw['B23025_002E'].astype(float)
unemployed = df_raw['B23025_005E'].astype(float)
below_pov = df_raw['B17001_002E'].astype(float)

df_raw['unemployment_rate'] = unemployed / labor_force.replace({0: np.nan})
df_raw['poverty_rate'] = below_pov / pop.replace({0: np.nan})

df_raw[['unemployment_rate', 'poverty_rate']].describe()



<a id="save-processed-data"></a>
## Save processed data

### Goal
Save a cleaned dataset to `data/processed/census_county_<year>.csv`.



### Your Turn (1): Save + reload


In [ ]:
out_path = PROCESSED_DIR / f'census_county_{year}.csv'
out_path.parent.mkdir(parents=True, exist_ok=True)

# TODO: Select a useful subset of columns and save.
# Suggested: NAME, state, county, raw vars, unemployment_rate, poverty_rate
cols = ['NAME', 'state', 'county'] + [c for c in acs_vars if c not in {'NAME'}] + ['unemployment_rate', 'poverty_rate']
df_out = df_raw[cols].copy()
df_out.to_csv(out_path, index=False)

df_check = pd.read_csv(out_path)
df_check.head()



### Checkpoint


In [ ]:
# TODO: Validate rates are in [0, 1] for most rows.
assert (df_out['unemployment_rate'].dropna().between(0, 1).mean() > 0.95)
assert (df_out['poverty_rate'].dropna().between(0, 1).mean() > 0.95)
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected file: data/processed/census_county_<year>.csv
# TODO: After saving your processed dataset, load it and run checks.
# df = pd.read_csv(PROCESSED_DIR / f'census_county_{year}.csv')
# assert df.shape[0] > 3000, "Expected ~3,200 counties"
# assert all(df['state'].astype(str).str.len() == 2), "State FIPS should be 2 chars"
# assert all(df['county'].astype(str).str.len() == 3), "County FIPS should be 3 chars"
# assert df['poverty_rate'].dropna().between(0, 1).all(), "Poverty rate out of bounds"
# print(df.dtypes)
...

## Extensions (Optional)
- Fetch an additional ACS table (e.g., B27001 for health insurance) and merge it with your county dataset on FIPS.
- Compare 1-year vs 5-year estimates for a large county — how much do they differ?
- Plot a choropleth or bar chart of poverty rates by state.


## Reflection
- Which counties had the highest margins of error relative to their estimates? Why?
- If you were merging this Census data with health outcomes (e.g., mortality rates), what FIPS code issues might you encounter?
- How would you handle a county that was renamed or re-coded between ACS vintages?


<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Browse variables</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_census_api_microdata_fetch — Browse variables
import json

# Offline default
print('Open the Census variables metadata in data/raw/census/variables_<year>.json if available.')
```

</details>

<details><summary>Solution: Fetch county data</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_census_api_microdata_fetch — Fetch county data
import pandas as pd

# Offline default sample
df = pd.read_csv(SAMPLE_DIR / 'census_county_sample.csv')
df.head()
```

</details>

<details><summary>Solution: Derived rates</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_census_api_microdata_fetch — Derived rates
df['unemployment_rate'] = df['B23025_005E'] / df['B23025_002E']
df['poverty_rate'] = df['B17001_002E'] / df['B01003_001E']
df[['unemployment_rate', 'poverty_rate']].describe()
```

</details>

<details><summary>Solution: Save processed data</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_census_api_microdata_fetch — Save processed data
from src import data as data_utils
year = 2022
data_utils.save_csv(df.set_index(['state','county'], drop=False), PROCESSED_DIR / f'census_county_{year}.csv')
print('saved')
```

</details>

